In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import warnings
#suppress warnings
warnings.filterwarnings('ignore')
print("Imports successful")

Imports successful


In [11]:
dataset = pd.read_csv(Path("../regressionAssignment/insurance_pre.csv"))
dataset = pd.get_dummies(dataset,drop_first=True)

independent_vars = ["age","bmi","children","sex_male","smoker_yes"]
dependent_var = "charges"

x = dataset[independent_vars]
y = dataset[dependent_var]

x.info()
print(f"x.shape={x.shape} y.shape={y.shape}")

X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=0)
print("created training and test dataset")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         1338 non-null   int64  
 1   bmi         1338 non-null   float64
 2   children    1338 non-null   int64  
 3   sex_male    1338 non-null   bool   
 4   smoker_yes  1338 non-null   bool   
dtypes: bool(2), float64(1), int64(2)
memory usage: 34.1 KB
x.shape=(1338, 5) y.shape=(1338,)
created training and test dataset


In [3]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = {
    'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
    'C': [10, 100, 1000, 2000, 3000],
    'gamma': ['auto', 'scale']
}

#refit if true will fit the model for best hyperparameter combination
grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=3, n_jobs=-1)

# fitting the model for grid search
grid.fit(X_train, y_train)


Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 5/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.102 total time=   0.0s
[CV 1/5] END .....C=10, gamma=auto, kernel=poly;, score=0.056 total time=   0.0s
[CV 3/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.104 total time=   0.0s
[CV 2/5] END .....C=10, gamma=auto, kernel=poly;, score=0.070 total time=   0.0s
[CV 2/5] END ......C=10, gamma=auto, kernel=rbf;, score=0.013 total time=   0.0s
[CV 1/5] END ......C=10, gamma=auto, kernel=rbf;, score=0.004 total time=   0.0s
[CV 4/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.025 total time=   0.0s
[CV 4/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.095 total time=   0.0s
[CV 5/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.050 total time=   0.0s
[CV 3/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.046 total time=   0.0s
[CV 2/5] END ..C=10, gamma=auto, kernel=sigmoid;, score=0.076 total time=   0.0s
[CV 4/5] END .C=10, gamma=auto, kernel=sigmoid;

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",SVR()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [10, 100, ...], 'gamma': ['auto', 'scale'], 'kernel': ['rbf', 'poly', ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",None
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 

In [5]:
#print best parameter
print(f"Best parameter={grid.best_params_}")

Best parameter={'C': 3000, 'gamma': 'scale', 'kernel': 'poly'}


In [6]:
#Let us see the results in tabular format
res_dict = grid.cv_results_
res_table = pd.DataFrame.from_dict(res_dict)

# Display only first 5 rows
print("First 5 rows:")
first_five_rows=res_table.head(5)
first_five_rows



First 5 rows:


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.026040,0.009309,0.008612,0.001616,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.004055,0.013366,-0.103821,-0.095119,-0.101604,-0.056625,0.053504,35
1,0.019260,0.006250,0.003881,0.001916,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.056274,0.069532,-0.045601,-0.025079,-0.049592,0.001107,0.051309,32
2,0.026338,0.005008,0.004101,0.000765,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.049905,0.075905,-0.046585,-0.041004,-0.046507,-0.001657,0.053391,34
3,0.018362,0.004086,0.002819,0.000996,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,25
4,0.016611,0.006592,0.008983,0.005721,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.004126,0.013244,-0.103775,-0.095165,-0.101602,-0.056634,0.053486,36


In [7]:
# Display the row that has rank_test_score = 1
# Note that hyper parameter combination 3000,scale,poly is ranked no 1

print("\nRow with rank_test_score = 1:")
res_table[res_table['rank_test_score'] == 1]



Row with rank_test_score = 1:


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
37,0.08167,0.036259,0.003025,0.00165,3000,scale,poly,"{'C': 3000, 'gamma': 'scale', 'kernel': 'poly'}",0.846874,0.805331,0.817283,0.805239,0.758988,0.806743,0.028307,1


In [8]:
#Set this to True for Vscode and False for Jupyter notebook
use_widgets = True

if use_widgets:
  from ipywidgets import widgets
  from IPython.display import display

  age_widget = widgets.IntText(description='Age:', value=0)
  bmi_widget = widgets.FloatText(description='BMI:', value=0.0)
  children_widget = widgets.IntText(description='Children:', value=0)
  sex_widget = widgets.Checkbox(description='Male?', value=False)
  smoker_widget = widgets.Checkbox(description='Smoker?', value=False)

  display(age_widget, bmi_widget, children_widget, sex_widget, smoker_widget)

else:
  print("Widgets not available, using input() instead")
  age = int(input("Enter Age: "))
  bmi = float(input("Enter BMI: "))
  children = int(input("No. of children: "))
  sex_male = input("Male? (yes/no): ").lower() in ['yes', 'y', '1']
  smoker = input("Smoker? (yes/no): ").lower() in ['yes', 'y', '1']  




IntText(value=0, description='Age:')

FloatText(value=0.0, description='BMI:')

IntText(value=0, description='Children:')

Checkbox(value=False, description='Male?')

Checkbox(value=False, description='Smoker?')

In [9]:
if use_widgets:
  age = age_widget.value
  bmi = bmi_widget.value
  children = children_widget.value
  sex_male = sex_widget.value
  smoker = smoker_widget.value
  
print(f"age={age} bmi={bmi} children={children} male={sex_male} smoker={smoker}")

age=25 bmi=22.5 children=2 male=True smoker=False


In [13]:
#Let us do prediction
transformed_input = sc.transform([[age,bmi,children,sex_male,smoker]])
prediction = grid.predict(transformed_input)

print(f"prediction={prediction}")

prediction=[3591.61515428]
